# EDA of the data
* Performing an EDA of the data to discover preliminary patterns and problems which we will use in our preprocessing of the data

# Importing required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder, MinMaxScaler
from datetime import datetime
from tqdm.notebook import tqdm #lets us display pretty progress bars for loops
import seaborn as sns
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import random #used for randomly selecting which plots to display (as there are a lot)

In C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\psydm7\Anaconda3\envs\mle_tf

# Setting up display options for pandas

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# Loading the data
* Try to load from pickle file as it is quicker to access
* If this fails, load the original CSV and save it to a pickle file
* <b>No point in doing anything with weather data as we only have 1 years worth of values so it is impossible to leverage with our chosen models to predict the next years worth of energy readings without imputing the 99% of missing values</b>

* Hence we are only looking into addInfo and consumption

In [3]:
#try to load pickle file as it is quicker
try:
    print("Loading consumption via pickle")
    df_energy = pd.read_pickle("../Data/Original_Data/consumption.pkl")
# if the file was not found then load the csv then save it to pickle for future loads
except:
    print("Loading consumption via csv")
    df_energy = pd.read_csv("../Data/Original_Data/consumption.csv")
    df_energy.to_pickle("../Data/Original_Data/consumption.pkl")

    #try to load pickle file as it is quicker
try:
    print("Loading addInfo via pickle")
    df_info = pd.read_pickle("../Data/Original_Data/addInfo.pkl")
# if the file was not found then load the csv then save it to pickle for future loads
except:
    print("Loading addInfo via csv")
    df_info = pd.read_csv("../Data/Original_Data/addInfo.csv")
    df_info.to_pickle("../Data/Original_Data/addInfo.pkl")

Loading consumption via pickle
Loading addInfo via pickle


# Inspecting df_energy and df_info

In [4]:
# inspecting the values of the dataframe
df_energy

,meter_id,2017-01-01 00:00:00,2017-01-01 00:30:00,2017-01-01 01:00:00,2017-01-01 01:30:00,2017-01-01 02:00:00,2017-01-01 02:30:00,2017-01-01 03:00:00,2017-01-01 03:30:00,2017-01-01 04:00:00,2017-01-01 04:30:00,2017-01-01 05:00:00,2017-01-01 05:30:00,2017-01-01 06:00:00,2017-01-01 06:30:00,2017-01-01 07:00:00,2017-01-01 07:30:00,2017-01-01 08:00:00,2017-01-01 08:30:00,2017-01-01 09:00:00,2017-01-01 09:30:00,2017-01-01 10:00:00,2017-01-01 10:30:00,2017-01-01 11:00:00,2017-01-01 11:30:00,...,2017-12-31 11:30:00,2017-12-31 12:00:00,2017-12-31 12:30:00,2017-12-31 13:00:00,2017-12-31 13:30:00,2017-12-31 14:00:00,2017-12-31 14:30:00,2017-12-31 15:00:00,2017-12-31 15:30:00,2017-12-31 16:00:00,2017-12-31 16:30:00,2017-12-31 17:00:00,2017-12-31 17:30:00,2017-12-31 18:00:00,2017-12-31 18:30:00,2017-12-31 19:00:00,2017-12-31 19:30:00,2017-12-31 20:00:00,2017-12-31 20:30:00,2017-12-31 21:00:00,2017-12-31 21:30:00,2017-12-31 22:00:00,2017-12-31 22:30:00,2017-12-31 23:00:00,2017-12-31 23:30:00
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.126,0.094,0.070,0.064,0.075,0.063,0.078,0.804,0.183,0.149,0.238,0.351,0.108,0.077,0.059,0.052,0.091,0.083,0.082,0.090,0.074,0.092,0.076,0.055,0.036
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.212,0.170,0.078,0.092,0.102,0.253,0.383,0.068,0.077,0.085,0.594,0.413,0.277,0.315,0.691,0.402,0.639,0.537,0.834,0.868,0.394,0.073,0.077,0.049,0.048
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005,0.029,0.196,0.134,0.069,0.067,0.025,0.026,0.020,0.014,0.032,0.019,0.107,0.104,0.124,0.110,0.123,0.012,0.034,0.006,0.027,0.001,0.044,0.004,0.035
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.104,0.039,0.120,0.063,0.078,0.111,0.074,0.103,0.635,0.421,0.466,0.087,0.052,0.115,0.150,0.118,0.096,0.135,0.041,0.018,0.057,0.018,0.038,0.040,0.014
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.545,0.150,0.498,1.165,1.054,0.498,0.199,0.058,0.019,0.068,0.077,0.060,0.024,0.026,0.016,0.022,0.010,0.036,0.006,0.024,0.014,0.028,0.006,0.024,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,0.011,1.089,0.014,0.204,0.015,0.159,0.014,0.013,0.174,0.013,0.014,0.014,0.014,0.189,0.015,0.011,0.011,0.011,0.011,0.011,0.011,0.011,0.011,0.011,...,0.195,0.181,0.177,0.184,0.198,0.177,0.182,0.180,0.196,0.182,0.177,0.181,0.203,0.176,0.183,0.176,0.200,0.184,0.177,0.182,0.201,0.181,0.181,0.177,0.198
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,0.015,1.523,0.105,0.296,0.067,0.171,0.025,0.260,0.013,0.024,0.012,0.178,0.018,0.454,0.290,1.228,1.228,1.229,1.199,1.075,0.021,0.009,0.019,0.011,...,0.009,0.008,0.016,0.028,0.008,0.009,0.009,0.015,0.028,0.009,0.012,0.023,0.034,0.040,0.025,0.022,0.009,0.028,0.020,0.010,0.009,0.008,0.033,0.015,0.013
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,0.222,0.276,0.097,0.058,0.045,0.045,0.038,0.034,0.043,0.044,0.046,0.033,0.038,0.043,0.056,0.250,0.187,0.207,0.182,0.164,0.144,0.055,0.052,0.051,...,0.181,0.195,0.462,0.631,0.203,0.189,0.242,0.264,0.187,0.479,0.316,0.275,0.503,0.422,0.343,0.144,0.169,0.145,0.200,0.170,0.127,0.135,0.200,0.118,0.106
3246,0x47218b46abb2fcaade487a211911406dc6e13730,0.195,2.881,1.765,1.391,1.609,1.362,1.352,1.321,0.701,0.784,0.587,0.788,0.636,0.728,0.097,0.061,0.177,0.097,0.136,0.128,0.114,0.109,0.152,0.147,...,0.116,0.126,0.118,0.136,0.126,0.137,0.138,0.158,0.220,0.215,0.511,0.717,0.647,0.653,0.330

In [5]:
# inspecting the shape of the dataframe
df_energy.shape

(3248, 17521)

In [6]:
# inspecting the values of the dataframe
df_info

,meter_id,dwelling_type,num_occupants,num_bedrooms,heating_fuel,hot_water_fuel,boiler_age,loft_insulation,wall_insulation,heating_temperature,efficient_lighting_percentage,dishwasher,freezer,fridge_freezer,refrigerator,tumble_dryer,washing_machine,game_console,laptop,pc,router,set_top_box,tablet,tv
0,0x02b7418b821b2edeeb2baa23ba1580566b4b0e43,semi_detached_house,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xaf32b2b497e9eb9c3f3de907aedc6a7914a11f41,terraced_house,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0xdd45ab28a7a40eadc04607d6da42e5ad923b1339,semi_detached_house,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x2b538a5ebfc3780aef25227267063cb6410d943b,detached_house,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0xee032ed498c9e99a4c65e0f39896fb30e513c6b7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,0xb9c600842cb2d195d461d675f2758ddbf5bb45bb,terraced_house,2.0,2.0,gas,gas,old,y,n,18_to_20,75_to_100,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0
2139,0x429e76a41f3141d9e62c4975e6b5df303cb45e64,bungalow,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2140,0x65fbd9ba598e703d742a32058ea2b323fe4c1fca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141,0x101f2dc409b3b95d2bb08ebe5d282f54317e3f8c,bungalow,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# inspecting the shape of the dataframe
df_info.shape

(2143, 24)

#### Conclusions to be drawn

* Can see that df_info only has entries for 2140 of the 3248 possible rows
* Must add the missing rows filled with NaN
    * Done in the EDA on AddInfo section where we simply join based on meter_id

# EDA on consumption

In [8]:
# how many missing values?
print(df_energy.isnull().sum().sum())

29378388


##### Conclusions:
* Large number of missing values hence we must find a way to deal with that
    * Will experiment with:
        1. Imputing the missing values linearly
        2. Dropping the missing values entirely and only using values we do have
        3. just treating missing values as 0
        
## Function for resampling into different time periods (will need this a lot)


* df_energy = the dataframe we are wanting to reschedule up
* file_name = name of file we will save it as
* resample_size = "D" or "M" or "W" corresponding to what period we are resampling into
* NaN_staregy
    * ignore : ignore nans and sum up rows regardless
    * removeColumn : if a nan is present in the column corresponding to the new column entry for this row, and set it to nan
    * imputeMean : if a nan is present in the columns corresponding to the new column entry for this row, treat them as the mean of the known values and sum up anyway for a reasonable estimate of what the total would have been

In [9]:
# only currently using this for days but may experiment with the months and weeks if have enough time
def resample_to_different_time(df_energy, file_name, resample_size, NaN_strategy):
    meter_id=df_energy["meter_id"]
    df_energy_new = pd.DataFrame(columns=["meter_id"])
    df_energy_new["meter_id"] = meter_id

    #for date of the resample size in the range of dates
    for new_sample in tqdm(pd.date_range(datetime(2017, 1, 1), datetime(2017, 12, 31), freq = resample_size),position=0):
        
        #get this columns name as a string
        if(resample_size=="M"):
            columnName = str(new_sample.date())[:7]
        elif(resample_size=="D" or resample_size=="W"):
            columnName = str(new_sample.date())
        else:
            return

        #if this isn't a week we can simply take those that start with the new columnName
        if(resample_size!="W"):
            #get all columns that relate to this new sample
            columns = [i for i in df_energy.columns.values[1:] if i.startswith(columnName)]
            
        #if it is a week be a bit more fancy find the days in the range of this week
        else:
            columns = []
            #for each day in range of this week
            for weeks_day in pd.date_range(new_sample, new_sample+timedelta(days=6), freq="D"):
                #get todays name as a string
                dayName = str(weeks_day.date())
                #get all columns that relate to this day
                dayColumns = [i for i in df_energy.columns.values[1:] if i.startswith(dayName)]
                #add it to the week total
                columns = columns+dayColumns
                
        #if we do not want to accept nans in our new columns
        if(NaN_strategy!="ignore"):
            #get series saying sum of nans in each row for the columns corresponding to this new column
            series_bool_contains_nulls = df_energy[columns].apply(lambda row: row.isnull().sum().sum(), axis=1)
            
            #for each rows boolean contains nulls truth value
            for row_id in tqdm(range(len(series_bool_contains_nulls)), position=1, leave=False):
                #this rows corresponding count of nulls
                num_contains_null=series_bool_contains_nulls[row_id]
                
                #if this row contains nulls....
                if(num_contains_null>1):
                    # if our nan strategy is setting the entire column to nan then do that
                    if(NaN_strategy=="removeColumn"):
                        #print(columnName,str(meter_id[row_id]),"(",str(row_id),") contains a null, setting this column to null")
                        df_energy_new.at[row_id,columnName]=np.nan
                        
                    # if our nan strategy is treating each nan as the groups mean then calculate the sum + add the mean * count of nums
                    # effectively imputing the nan entries as the mean to get a reasonable guess of what the total is
                    elif(NaN_strategy=="imputeMean"):
                        rowMean = df_energy[columns].iloc[row_id].mean(skipna=True)
                        #print(columnName,str(meter_id[row_id]),"(",str(row_id),") contains",num_contains_null,"null\n",num_contains_null,"*",rowMean,"=",rowMean*num_contains_null)
                        df_energy_new.at[row_id,columnName]=(df_energy[columns].iloc[row_id].sum()+(rowMean * num_contains_null))
                    
                    # else we aren't using a valid nan strategy so return
                    else:
                        return
                        
                #else this row doesn't contain nulls set it's corresponding cell in the new column to the sum
                else:
                    df_energy_new.at[row_id,columnName]=df_energy[columns].iloc[row_id].sum()

        #else we don't care if it contains nans or not so just sum up these columns to our new day    
        else:
            #sum these up into a value for the new sample size
            df_energy_new[columnName] = df_energy[columns].sum(axis=1)
    
    #if we leave file name blank we don't want to save
    if(file_name!=""):
        #saving the dataframe
        #if(os.path.exists(f"../Data/Preprocessed_Data/{file_name}.pkl")==False):
        #    df_energy_new.to_pickle(f"../Data/Preprocessed_Data/{file_name}.pkl")
        df_energy_new.to_pickle(f"../Data/Preprocessed_Data/{file_name}.pkl")
    return df_energy_new

## Resampling from 30 min intervals into days 
* Resmapling into days as it'll be better suited for our RNN based models to learn from
    * These are what we will actually predict 
        * Eg will train on to use the prior 14 time steps (days) to predict the next 7 time steps (days)
            * Then will iteratively use those predictions to prediction the next 7 days: until we have done all 365 and then aggregate those predictions into the monthly sums ready for submission
            
            
* TODO : Experiment on if I should accept NaNs or not currently set to True
    * Ended up setting to false with daily energy that isn't interpolated will take a hard stance that anything that has missing is considered unreliable and thus will just not consider it at all

In [10]:
# resampling the original data into a daily value setting days that contain a missing to be missing
df_energy_daily = resample_to_different_time(df_energy, "consumption_daily_nan_removeColumn","D","removeColumn")

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

...

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

In [11]:
df_energy_daily

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-07,2017-12-08,2017-12-09,2017-12-10,2017-12-11,2017-12-12,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.471,3.741,NaN,7.224,4.951,NaN,3.442,3.509,4.675,6.131,4.962,3.336,4.318,3.561,4.279,4.030,5.397,NaN,4.818,3.931,NaN,4.503,NaN,5.129,5.395
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.251,23.469,NaN,24.730,24.597,NaN,20.056,22.546,22.333,23.353,22.774,20.229,21.218,15.419,13.475,13.101,14.327,NaN,14.936,16.174,NaN,24.618,NaN,15.167,11.751
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.360,18.557,NaN,12.423,18.379,NaN,13.281,13.821,12.755,16.712,12.364,16.055,18.562,10.385,10.917,10.201,7.320,NaN,7.384,14.425,NaN,17.705,NaN,8.966,4.633
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.110,23.315,NaN,19.672,21.023,NaN,19.686,23.435,18.733,22.249,20.755,19.730,21.135,16.526,12.677,14.833,12.477,NaN,10.974,19.646,NaN,23.993,NaN,15.841,14.452
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.507,35.388,NaN,48.607,54.760,NaN,42.703,35.045,38.664,36.790,29.009,42.716,29.052,25.444,28.234,39.440,35.538,NaN,8.351,9.957,NaN,25.871,NaN,46.274,16.901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,2.317,2.301,2.352,2.516,2.229,2.354,2.397,2.397,2.269,NaN,2.339,2.212,2.456,2.229,2.503,2.207,2.469,2.266,2.321,2.262,2.286,2.384,2.319,2.289,...,9.675,9.976,NaN,10.635,10.650,NaN,10.149,10.112,10.632,10.566,10.732,10.615,10.386,9.936,9.625,9.390,9.231,NaN,9.216,9.336,NaN,10.032,NaN,9.757,9.480
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,14.437,16.274,7.031,17.018,17.603,15.005,8.987,8.490,10.136,NaN,10.466,8.177,11.929,8.261,12.369,14.352,14.058,10.472,17.396,9.879,21.971,12.708,15.327,8.309,...,3.598,5.862,NaN,7.770,5.091,NaN,6.109,5.056,3.972,3.458,7.793,9.837,8.697,5.079,6.031,4.141,2.828,NaN,4.818,3.357,NaN,12.906,NaN,3.122,3.401
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,7.824,7.517,5.398,6.788,7.360,6.898,7.321,8.042,8.207,NaN,5.464,7.545,6.432,6.001,5.558,6.391,6.066,6.443,6.675,7.363,7.269,7.183,7.102,NaN,...,5.735,7.241,NaN,5.695,8.143,NaN,6.993,6.451,7.629,6.880,5.960,6.231,5.748,5.259,5.258,6.767,5.919,NaN,6.077,7.761,NaN,5.455,NaN,5.679,8.148
3246,0x47218b46abb2fcaade487a211911406dc6e13730,23.965,28.689,27.664,29.229,29.548,27.909,26.923,21.277,23.452,NaN,25.924,26.834,30.346,30.367,26.135,26.842,25.588,30.267,28.921,27.530,28.424,28.961,29.346,28.640,...,23.870,28.726,NaN,29.844,31.899,NaN,30.039,30.470,30.044,28.617,31.578,31.481,32.123,29.524,22.586,20.747,19.979,NaN,20.406,23.668,NaN,31.912,NaN,26.313,24.201


In [12]:
# resampling the original data into a daily value setting days that ignores missing and still sums the day 
df_energy_daily_with_missing = resample_to_different_time(df_energy, "consumption_daily_nan_ignored","D","ignore")

  0%|          | 0/365 [00:00<?, ?it/s]

In [13]:
df_energy_daily_with_missing

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-07,2017-12-08,2017-12-09,2017-12-10,2017-12-11,2017-12-12,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,4.471,3.741,0.0,7.224,4.951,0.0,3.442,3.509,4.675,6.131,4.962,3.336,4.318,3.561,4.279,4.030,5.397,0.0,4.818,3.931,0.0,4.503,0.0,5.129,5.395
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,17.251,23.469,0.0,24.730,24.597,0.0,20.056,22.546,22.333,23.353,22.774,20.229,21.218,15.419,13.475,13.101,14.327,0.0,14.936,16.174,0.0,24.618,0.0,15.167,11.751
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,6.360,18.557,0.0,12.423,18.379,0.0,13.281,13.821,12.755,16.712,12.364,16.055,18.562,10.385,10.917,10.201,7.320,0.0,7.384,14.425,0.0,17.705,0.0,8.966,4.633
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,14.110,23.315,0.0,19.672,21.023,0.0,19.686,23.435,18.733,22.249,20.755,19.730,21.135,16.526,12.677,14.833,12.477,0.0,10.974,19.646,0.0,23.993,0.0,15.841,14.452
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,30.507,35.388,0.0,48.607,54.760,0.0,42.703,35.045,38.664,36.790,29.009,42.716,29.052,25.444,28.234,39.440,35.538,0.0,8.351,9.957,0.0,25.871,0.0,46.274,16.901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,2.317,2.301,2.352,2.516,2.229,2.354,2.397,2.397,2.269,0.0,2.339,2.212,2.456,2.229,2.503,2.207,2.469,2.266,2.321,2.262,2.286,2.384,2.319,2.289,...,9.675,9.976,0.0,10.635,10.650,0.0,10.149,10.112,10.632,10.566,10.732,10.615,10.386,9.936,9.625,9.390,9.231,0.0,9.216,9.336,0.0,10.032,0.0,9.757,9.480
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,14.437,16.274,7.031,17.018,17.603,15.005,8.987,8.490,10.136,0.0,10.466,8.177,11.929,8.261,12.369,14.352,14.058,10.472,17.396,9.879,21.971,12.708,15.327,8.309,...,3.598,5.862,0.0,7.770,5.091,0.0,6.109,5.056,3.972,3.458,7.793,9.837,8.697,5.079,6.031,4.141,2.828,0.0,4.818,3.357,0.0,12.906,0.0,3.122,3.401
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,7.824,7.517,5.398,6.788,7.360,6.898,7.321,8.042,8.207,0.0,5.464,7.545,6.432,6.001,5.558,6.391,6.066,6.443,6.675,7.363,7.269,7.183,7.102,0.000,...,5.735,7.241,0.0,5.695,8.143,0.0,6.993,6.451,7.629,6.880,5.960,6.231,5.748,5.259,5.258,6.767,5.919,0.0,6.077,7.761,0.0,5.455,0.0,5.679,8.148
3246,0x47218b46abb2fcaade487a211911406dc6e13730,23.965,28.689,27.664,29.229,29.548,27.909,26.923,21.277,23.452,0.0,25.924,26.834,30.346,30.367,26.135,26.842,25.588,30.267,28.921,27.530,28.424,28.961,29.346,28.640,...,23.870,28.726,0.0,29.844,31.899,0.0,30.039,30.470,30.044,28.617,31.578,31.481,32.123,29.524,22.586,20.747,19.979,0.0,20.406,23.668,0.0,31.912,0.0,26.313,24.201


## Linearly interpolating the missing values from 30 min then resampling into days

* Linearly interpolate the half hours with a limit of 4 hours worth of consecutive missing values
* Then aggregate into days setting days containing a missing to the entire day as missing
* Then interpolating those missing days

- Experiment with different limits for how many consecutive nans are okay to fit 
    - current set to 8 (4 hours worth)

In [14]:
# how many consecutive NaNs to fill
seq_nan_limit = 8

In [15]:
#dropping the meter id column for now so we can interpolate
meter_id = df_energy["meter_id"]
df_energy_interpolate = df_energy.drop(["meter_id"], axis=1)

In [16]:
#replacing 0 back to NaN
df_energy_interpolate=df_energy_interpolate.replace(0,np.nan)

In [17]:
# interpolating the missing values
df_energy_interpolate = df_energy_interpolate.interpolate(method="linear", axis=1, limit = seq_nan_limit) 
# adding the meter id back on
df_energy_interpolate = pd.concat([meter_id,df_energy_interpolate],axis=1)

In [18]:
df_energy_interpolate

,meter_id,2017-01-01 00:00:00,2017-01-01 00:30:00,2017-01-01 01:00:00,2017-01-01 01:30:00,2017-01-01 02:00:00,2017-01-01 02:30:00,2017-01-01 03:00:00,2017-01-01 03:30:00,2017-01-01 04:00:00,2017-01-01 04:30:00,2017-01-01 05:00:00,2017-01-01 05:30:00,2017-01-01 06:00:00,2017-01-01 06:30:00,2017-01-01 07:00:00,2017-01-01 07:30:00,2017-01-01 08:00:00,2017-01-01 08:30:00,2017-01-01 09:00:00,2017-01-01 09:30:00,2017-01-01 10:00:00,2017-01-01 10:30:00,2017-01-01 11:00:00,2017-01-01 11:30:00,...,2017-12-31 11:30:00,2017-12-31 12:00:00,2017-12-31 12:30:00,2017-12-31 13:00:00,2017-12-31 13:30:00,2017-12-31 14:00:00,2017-12-31 14:30:00,2017-12-31 15:00:00,2017-12-31 15:30:00,2017-12-31 16:00:00,2017-12-31 16:30:00,2017-12-31 17:00:00,2017-12-31 17:30:00,2017-12-31 18:00:00,2017-12-31 18:30:00,2017-12-31 19:00:00,2017-12-31 19:30:00,2017-12-31 20:00:00,2017-12-31 20:30:00,2017-12-31 21:00:00,2017-12-31 21:30:00,2017-12-31 22:00:00,2017-12-31 22:30:00,2017-12-31 23:00:00,2017-12-31 23:30:00
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.126,0.094,0.070,0.064,0.075,0.063,0.078,0.804,0.183,0.149,0.238,0.351,0.108,0.077,0.059,0.052,0.091,0.083,0.082,0.090,0.074,0.092,0.076,0.055,0.036
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.212,0.170,0.078,0.092,0.102,0.253,0.383,0.068,0.077,0.085,0.594,0.413,0.277,0.315,0.691,0.402,0.639,0.537,0.834,0.868,0.394,0.073,0.077,0.049,0.048
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005,0.029,0.196,0.134,0.069,0.067,0.025,0.026,0.020,0.014,0.032,0.019,0.107,0.104,0.124,0.110,0.123,0.012,0.034,0.006,0.027,0.001,0.044,0.004,0.035
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.104,0.039,0.120,0.063,0.078,0.111,0.074,0.103,0.635,0.421,0.466,0.087,0.052,0.115,0.150,0.118,0.096,0.135,0.041,0.018,0.057,0.018,0.038,0.040,0.014
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.545,0.150,0.498,1.165,1.054,0.498,0.199,0.058,0.019,0.068,0.077,0.060,0.024,0.026,0.016,0.022,0.010,0.036,0.006,0.024,0.014,0.028,0.006,0.024,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,0.011,1.089,0.014,0.204,0.015,0.159,0.014,0.013,0.174,0.013,0.014,0.014,0.014,0.189,0.015,0.011,0.011,0.011,0.011,0.011,0.011,0.011,0.011,0.011,...,0.195,0.181,0.177,0.184,0.198,0.177,0.182,0.180,0.196,0.182,0.177,0.181,0.203,0.176,0.183,0.176,0.200,0.184,0.177,0.182,0.201,0.181,0.181,0.177,0.198
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,0.015,1.523,0.105,0.296,0.067,0.171,0.025,0.260,0.013,0.024,0.012,0.178,0.018,0.454,0.290,1.228,1.228,1.229,1.199,1.075,0.021,0.009,0.019,0.011,...,0.009,0.008,0.016,0.028,0.008,0.009,0.009,0.015,0.028,0.009,0.012,0.023,0.034,0.040,0.025,0.022,0.009,0.028,0.020,0.010,0.009,0.008,0.033,0.015,0.013
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,0.222,0.276,0.097,0.058,0.045,0.045,0.038,0.034,0.043,0.044,0.046,0.033,0.038,0.043,0.056,0.250,0.187,0.207,0.182,0.164,0.144,0.055,0.052,0.051,...,0.181,0.195,0.462,0.631,0.203,0.189,0.242,0.264,0.187,0.479,0.316,0.275,0.503,0.422,0.343,0.144,0.169,0.145,0.200,0.170,0.127,0.135,0.200,0.118,0.106
3246,0x47218b46abb2fcaade487a211911406dc6e13730,0.195,2.881,1.765,1.391,1.609,1.362,1.352,1.321,0.701,0.784,0.587,0.788,0.636,0.728,0.097,0.061,0.177,0.097,0.136,0.128,0.114,0.109,0.152,0.147,...,0.116,0.126,0.118,0.136,0.126,0.137,0.138,0.158,0.220,0.215,0.511,0.717,0.647,0.653,0.330

In [19]:
#resampling to days
df_energy_interpolate_days = resample_to_different_time(df_energy_interpolate, "","D","removeColumn")

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

...

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

  0%|          | 0/3248 [00:00<?, ?it/s]

In [20]:
df_energy_interpolate_days

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-07,2017-12-08,2017-12-09,2017-12-10,2017-12-11,2017-12-12,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.4710,3.7410,NaN,7.224,4.951,NaN,3.4420,3.509,4.6750,6.131,4.962,3.336,4.318,3.561,4.279,4.030,5.397,NaN,4.818,3.931,NaN,4.503,NaN,5.129,5.395
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.2510,23.4690,NaN,24.730,24.597,NaN,20.0560,22.546,22.3330,23.353,22.774,20.229,21.218,15.419,13.475,13.101,14.327,NaN,14.936,16.174,NaN,24.618,NaN,15.167,11.751
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3985,20.0265,NaN,12.423,18.379,NaN,13.3215,13.821,14.2175,16.712,12.364,16.102,18.601,10.385,10.917,10.201,7.320,NaN,7.384,14.425,NaN,17.705,NaN,8.966,4.633
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.1100,23.3150,NaN,19.672,21.023,NaN,19.6860,23.435,18.7330,22.249,20.755,19.730,21.135,16.526,12.677,14.833,12.477,NaN,10.974,19.646,NaN,23.993,NaN,15.841,14.452
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.5070,35.3880,NaN,48.607,54.760,NaN,42.7030,35.045,38.6640,36.790,29.009,42.716,29.052,25.444,28.234,39.440,35.538,NaN,8.351,9.957,NaN,25.871,NaN,46.274,16.901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,2.317,2.301,2.352,2.516,2.229,2.354,2.397,2.397,2.269,NaN,2.339,2.212,2.456,2.229,2.503,2.207,2.469,2.266,2.321,2.262,2.286,2.384,2.319,2.289,...,9.6750,9.9760,NaN,10.635,10.650,NaN,10.1490,10.112,10.6320,10.566,10.732,10.615,10.386,9.936,9.625,9.390,9.231,NaN,9.216,9.336,NaN,10.032,NaN,9.757,9.480
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,14.437,16.274,7.031,17.018,17.603,15.005,8.987,8.490,10.136,NaN,10.466,8.177,11.929,8.261,12.369,14.352,14.058,10.472,17.396,9.879,21.971,12.708,15.327,8.309,...,3.5980,5.8620,NaN,7.770,5.091,NaN,6.1090,5.056,3.9720,3.458,7.793,9.837,8.697,5.079,6.031,4.141,2.828,NaN,4.818,3.357,NaN,12.906,NaN,3.122,3.401
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,7.824,7.517,5.398,6.788,7.360,6.898,7.321,8.042,8.207,NaN,5.464,7.545,6.432,6.001,5.558,6.391,6.066,6.443,6.675,7.363,7.269,7.183,7.102,NaN,...,5.7350,7.2410,NaN,5.695,8.143,NaN,6.9930,6.451,7.6290,6.880,5.960,6.231,5.748,5.259,5.258,6.767,5.919,NaN,6.077,7.761,NaN,5.455,NaN,5.679,8.148
3246,0x47218b46abb2fcaade487a211911406dc6e13730,23.965,28.689,27.664,29.229,29.548,27.909,26.923,21.277,23.452,NaN,25.924,26.834,30.346,30.367,26.135,26.842,25.588,30.267,28.921,27.530,28.424,28.961,29.346,28.640,...,23.8700,28.7260,NaN,29.844,31.899,NaN,30.0390,30.470,30.0440,28.617,31.578,31.481,32.123,29.524,22.586,20.747,19.979,NaN,20.406,23.668,NaN,31.912,NaN,26.313,24.201


In [21]:
#replacing 0 with nan
df_energy_interpolate_days_interpolate = df_energy_interpolate_days.replace(0, np.nan)

In [22]:
#dropping meter_id
meter_id = df_energy_interpolate_days_interpolate["meter_id"] 
df_energy_interpolate_days_interpolate=df_energy_interpolate_days_interpolate.drop("meter_id", axis=1)
#interpolating the days we set to nan due to containing a missing value as acceptNaNs is false
df_energy_interpolate_days_interpolate = df_energy_interpolate_days_interpolate.interpolate(method="linear", axis=1) 
#adding meter_id back on
df_energy_interpolate_days_interpolate = pd.concat([pd.DataFrame(meter_id), df_energy_interpolate_days_interpolate],axis=1)

In [23]:
df_energy_interpolate_days_interpolate

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-07,2017-12-08,2017-12-09,2017-12-10,2017-12-11,2017-12-12,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.4710,3.7410,5.48250,7.224,4.951,4.19650,3.4420,3.509,4.6750,6.131,4.962,3.336,4.318,3.561,4.279,4.030,5.397,5.1075,4.818,3.931,4.2170,4.503,4.8160,5.129,5.395
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.2510,23.4690,24.09950,24.730,24.597,22.32650,20.0560,22.546,22.3330,23.353,22.774,20.229,21.218,15.419,13.475,13.101,14.327,14.6315,14.936,16.174,20.3960,24.618,19.8925,15.167,11.751
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3985,20.0265,16.22475,12.423,18.379,15.85025,13.3215,13.821,14.2175,16.712,12.364,16.102,18.601,10.385,10.917,10.201,7.320,7.3520,7.384,14.425,16.0650,17.705,13.3355,8.966,4.633
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.1100,23.3150,21.49350,19.672,21.023,20.35450,19.6860,23.435,18.7330,22.249,20.755,19.730,21.135,16.526,12.677,14.833,12.477,11.7255,10.974,19.646,21.8195,23.993,19.9170,15.841,14.452
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.5070,35.3880,41.99750,48.607,54.760,48.73150,42.7030,35.045,38.6640,36.790,29.009,42.716,29.052,25.444,28.234,39.440,35.538,21.9445,8.351,9.957,17.9140,25.871,36.0725,46.274,16.901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,2.317,2.301,2.352,2.516,2.229,2.354,2.397,2.397,2.269,2.3040,2.339,2.212,2.456,2.229,2.503,2.207,2.469,2.266,2.321,2.262,2.286,2.384,2.319,2.289,...,9.6750,9.9760,10.30550,10.635,10.650,10.39950,10.1490,10.112,10.6320,10.566,10.732,10.615,10.386,9.936,9.625,9.390,9.231,9.2235,9.216,9.336,9.6840,10.032,9.8945,9.757,9.480
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,14.437,16.274,7.031,17.018,17.603,15.005,8.987,8.490,10.136,10.3010,10.466,8.177,11.929,8.261,12.369,14.352,14.058,10.472,17.396,9.879,21.971,12.708,15.327,8.309,...,3.5980,5.8620,6.81600,7.770,5.091,5.60000,6.1090,5.056,3.9720,3.458,7.793,9.837,8.697,5.079,6.031,4.141,2.828,3.8230,4.818,3.357,8.1315,12.906,8.0140,3.122,3.401
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,7.824,7.517,5.398,6.788,7.360,6.898,7.321,8.042,8.207,6.8355,5.464,7.545,6.432,6.001,5.558,6.391,6.066,6.443,6.675,7.363,7.269,7.183,7.102,6.449,...,5.7350,7.2410,6.46800,5.695,8.143,7.56800,6.9930,6.451,7.6290,6.880,5.960,6.231,5.748,5.259,5.258,6.767,5.919,5.9980,6.077,7.761,6.6080,5.455,5.5670,5.679,8.148
3246,0x47218b46abb2fcaade487a211911406dc6e13730,23.965,28.689,27.664,29.229,29.548,27.909,26.923,21.277,23.452,24.6880,25.924,26.834,30.346,30.367,26.135,26.842,25.588,30.267,28.921,27.530,28.424,28.961,29.346,28.640,...,23.8700,28.7260,29.28500,29.844,31.899,30.96900,30.0390,30.470,30.0440,28.617,31.578,31.481,32.123,29.524,22.586,20.747,19.979,20.1925,20.406,23.668,27.7900,31.912,29.1125,26.313,24.201


In [24]:
# saving the dataframe
#if(os.path.exists("../Data/Preprocessed_Data/consumption_daily_nan_interpolated.pkl")==False):
#        df_energy_interpolate_days_interpolate.to_pickle("../Data/Preprocessed_Data/consumption_daily_nan_interpolated.pkl")
df_energy_interpolate_days_interpolate.to_pickle("../Data/Preprocessed_Data/consumption_daily_nan_interpolated.pkl")

## Plotting graph of the time patterns for each meter

time_step_size =
* O : Original
* D : Day

In [25]:
def plot_time_patterns(df_energy_day, directory, show_pid, time_step_size):
      
    df_energy_day_plot = df_energy_day.drop(["meter_id"], axis=1)
   
    for pid in tqdm(range(0,3248)):
        
        #getting this row
        meter_id = df_energy_day.iloc[pid,0]
        df_days_pid = df_energy_day_plot[pid:pid+1].T

        #converting index to datetime for ease of plots key
        df_days_pid.index=pd.to_datetime(df_days_pid.index)
        
        #creating figure
        fig = plt.figure(figsize=(20,10))
        
        # if this is already in days just plot the days with the weekly and monthly moving averages
        if(time_step_size == "D"):
            #calculating the week and month using moving averages
            month_average=df_days_pid.rolling(30).mean()
            #plot the monthly moving moving average and the daily readings
            plt.plot(month_average, label="monthly moving average", lw=2, color="darkorange")
            plt.plot(df_days_pid, label="total daily energy", lw=1,color="skyblue")
        
        #elif this is the original half an hour readings also add a separate moving average for the day
        elif(time_step_size == "O"):
            #calculating the daily moving average (48 half hours)
            day_average=df_days_pid.rolling(48).mean()
            #plotting half hourly energy and daily moving average
            plt.plot(day_average, label="daily moving average, lw=0.5,color="darkorange")
            plt.plot(df_days_pid, label="total half hourly energy", lw=0.2,color="skyblue")
                     
        #else it's uncrecognised so return
        else:
            return
            
        #annotations
        plt.legend(fontsize=20)
        plt.title("Meter '"+str(meter_id)+"' energy readings", fontsize=15)
        plt.xlabel("Date")
        plt.ylabel("Energy Usage kWh")
        
        #showing a subset of the plots
        if(pid in show_pid):
            print("showing row: ",pid)
            plt.show()
        
        #saving plot if it doesn't already exist
        #if(os.path.exists(f"../EDA/plots/consumption/{directory}/energy_usage_{pid}_{meter_id}.png")==False):
        #    fig.savefig(f"../EDA/plots/consumption/{directory}/energy_usage_{pid}_{meter_id}.png")
        fig.savefig(f"../EDA/plots/consumption/{directory}/energy_usage_{pid}_{meter_id}.png")
        
        plt.close()

SyntaxError: invalid syntax (<ipython-input-25-d15714b74a52>, line 30)

In [ ]:
#getting plot id's we want to show (random amount as no point in showing 9000 plots in compiler it'll make it unnecessarily slow)
#display_pid = random.sample(range(0,3248),showPlots) # new randomly selected sample
#display_pid = [651, 3034, 1346, 1618, 2105] # previously randomly selected sample which we can repeat

display_pid=[849,2410,2676,2680,2965,3158]#meters which contained values from within the max sequential nan limit

#display_pid=[]#empty for not displaying any
display_pid

In [ ]:
#plotting the original data
plot_time_patterns(df_energy.replace(np.nan,0), "Original", display_pid, "O")

In [ ]:
#plotting the uninterpolated data with days with missing still summed to a daily value (but inaccurate as some hours missing)
plot_time_patterns(df_energy_daily_with_missing.replace(np.nan, 0), "Daily_Ignored", display_pid, "D")

In [ ]:
#plotting the uninterpolated data with days with missing set to missing
plot_time_patterns(df_energy_daily.replace(np.nan, 0), "Daily_Removed", display_pid, "D")

In [ ]:
#plotting the interpolated data
plot_time_patterns(df_energy_interpolate_days_interpolate.replace(np.nan,0), "Daily_Interpolated", display_pid, "D")

## Comparing how many missing values are now missing

In [ ]:
print("Interpolated dataset has this many missing values")
print(df_energy_interpolate_days_interpolate.replace(0, np.nan).isna().sum().sum())

In [ ]:
print("Uninterpolated dataset with days with missing values set to missing has this many missing values")
print(df_energy_daily.replace(0, np.nan).isna().sum().sum())

In [ ]:
print("Uninterpolated dataset with days with missing values set to sum of values it does have has this many missing values")
print(df_energy_daily_with_missing.replace(0, np.nan).isna().sum().sum())

##### Conclusion
* Essentially can conclude interpolated has successfully done a good job at imputing the missing values
    * Has similar pattern to that of the summed with missing
        * But more accurate and much fewer missing values
* Will use this dataset in the future for that reason
* Now only has the 'prefix nans' still missing (nans from before the meters started)
    * And some 'sufix na' (when meter failed and stopped recording
        * Will need more EDA to find where these are and handle them
            * Perhaps just write a script checking % variance and if it's at unexpected level then it's possibly failed?
                * Eg this value is 99% lower than the yearly average (probably means something went wrong)v


* Have removed the missing values that could be removed via linear interpolation:
    * when there are values before and after the sequence of ?
* Still have lots of missing values at start and end of dataset
    * IN the data preparation for RNN and feature engineering stage will add a way to deal with this as mentioned perhaps checking if it falls under a certain threshold and then classifying the meter as not working that day if it was under that threshold and then set the day as missing
        * As it'll be atypical


_________
_________________________________________________________
_________


# EDA on AddInfo
## Adding missing rows to AddInfo based on missing meter_id's

In [ ]:
# first making both dataframes alphabetical in order of meter_id for ease of comparison
df_info.sort_values('meter_id')

In [ ]:
df_energy.sort_values('meter_id')

In [ ]:
#adding the missing meter rows to df_info
df_info_all_meters = pd.merge(df_info, df_energy["meter_id"], on="meter_id",how="outer")

In [ ]:
#checking the values are correct now
df_info_all_meters.sort_values('meter_id')

In [ ]:
#checking the shape is correct now
df_info_all_meters.shape

## Exploring datatypes and unique values

In [ ]:
# inspecting the types of each column
df_info_all_meters.dtypes

In [ ]:
#looking at the unique values for each column
for column in df_info_all_meters.drop(["meter_id"],axis=1).columns:
    print(column+" unique value count: "+str(df_info_all_meters[column].nunique()))
    print(df_info_all_meters[column].unique())
    print()

* We now know:
    1. The columns that contain categorical values vs the columns that contain real numbers
    2. The unique values for each column
    
## Exploring missing values
### Visualising as histogram

In [ ]:
# plotting a histogram for each of the columns in the dataset
for column in df_info_all_meters.drop(["meter_id"],axis=1).columns:
    df_temp = df_info_all_meters[column].replace(np.nan,"NaN").apply(str)
    ax = df_temp.hist(figsize=(10,10))
    ax.set_xlabel("unique values")
    ax.set_ylabel("count")
    fig = ax.get_figure()
    plt.title(column+" values")
    plt.show(block=False)
    
#     if(os.path.exists(f"../EDA/plots/addInfo/{column}_hist.png")==False):
#         fig.savefig(f"../EDA/plots/addInfo/{column}_hist.png")
    fig.savefig(f"../EDA/plots/addInfo/{column}_hist.png")
            
    plt.close(fig)

### Calculating percentage of missing values for each column

In [ ]:
# Calculating percentage of missing values
#stores index of columns to keep: defaults all to true and will check which to make false
columns = np.full((df_info_all_meters.shape[1],), True, dtype=bool)

# getting the count of each object type in each column
for i in range(0,len(df_info_all_meters.columns)):
    column =df_info_all_meters.columns[i]
    #calculating the percentage of missing values
    percent_missing = df_info_all_meters[column].isnull().sum() * 100 / len(df_info_all_meters)
    print(column+" contains missing values: "+str(df_info_all_meters[column].isna().values.any()))
    print(column+" is missing: "+str(round(percent_missing,2))+"%")
    
    #if the percentage of missing values is greater than 50% mark them to be removed
    if(percent_missing>=50):
        print("Marked ",column,"for removal due to it having more than 50% missing values")
        columns[i] = False
    else:
        print("Not marked ",column,"for removal due to it not having more than 50% missing values")
    print()

#### Meaning of findings;
* As the histograms and calculations show: there is a massive amount of missing values
* Marked all columns with > 50% missing values for removal
    * as if more than half of the values are missing it is likely what we do impute won't be accurate
* Need to decide what to do with the the remaining 2 as they have 40% missing values: is this good enough?
    * Will likely require experimentation:
        * So will run one model which only uses energy
        * And one that uses addInfo after the missing values have been imputed

## Removing the columns with greater than 50% missing values

In [ ]:
# Removing the columns marked for removal
selected_columns = df_info_all_meters.columns[columns]
df_info_all_meters = df_info_all_meters[selected_columns]

In [ ]:
# inspecting the current state of df_info_all_meters
df_info_all_meters

## Imputing the missing values
* May be worth experimenting with different imputation methods
* But for now just going with KNN

### Preparing dataframe for use by KNNImputer
### Adding information on the energy time series data that may be useful for imputing the missing addInfo values
Experimenting with different combinations 9think it may have been too many when I originally used 4 different energy ones and it got weighted too highly skewing it due to the multi collinearity problems

In [ ]:
# Calculating the total, mean, max, and min energy usage only using KNOWN values (not including the interpolated ones)
df_energy_info = pd.DataFrame(columns=["meter_id",
                                       "mean_energy_usage",
                                       #"max_energy_usage",
                                       #"min_energy_usage",
                                       "total_energy_usage"
                                      ])
df_energy_info["meter_id"]=df_energy_interpolate_days_interpolate["meter_id"].values
df_energy_info["mean_energy_usage"] = df_energy_interpolate_days_interpolate.replace(0,np.nan).iloc[:, 1:].mean(axis=1).round(3)
#df_energy_info["max_energy_usage"] = df_energy_interpolate_days_interpolate.replace(0,np.nan).iloc[:, 1:].max(axis=1).round(3)
#df_energy_info["min_energy_usage"] = df_energy_interpolate_days_interpolate.replace(0,np.nan).iloc[:, 1:].min(axis=1).round(3)
df_energy_info["total_energy_usage"] = df_energy_interpolate_days_interpolate.replace(0,np.nan).iloc[:, 1:].sum(axis=1).round(3)

In [ ]:
df_energy_info#.loc[df_energy_means["meter_id"]=="0xa62b9f23553ff183f61e2bf943aab3d5983d02d7"]

In [ ]:
#add the total, mean, max, and min to the df for use with knn imputation
df_info_knn = df_info_all_meters
df_info_knn = df_info_knn.merge(df_energy_info[["meter_id",
                                                "mean_energy_usage",
                                                #"max_energy_usage",
                                                #"min_energy_usage", 
                                                "total_energy_usage"
                                               ]], on="meter_id",how="left")
df_info_knn#.loc[df_info_knn2["meter_id"]=="0xa62b9f23553ff183f61e2bf943aab3d5983d02d7"]

### Dropping meter_id, encoding and normalizing data so it is suitable for KNN Imputation
#### Dropping meter_id

In [ ]:
#dropping the id column so it doesn't negatively influence the knn algorithm
meter_id = df_info_knn["meter_id"]
df_info_knn = df_info_knn.drop(["meter_id"], axis=1)

In [ ]:
#inspecting the df without the id column
df_info_knn

#### Encoding the categorical values ready for knn

In [ ]:
df_info_knn_encoded = df_info_knn
df_info_knn_encoded["dwelling_type"]=df_info_knn_encoded["dwelling_type"].replace("flat",0).replace("terraced_house",1).replace("semi_detached_house",2).replace("detached_house",3).replace("bungalow",4)

In [ ]:
df_info_knn_encoded

#### Normalizing the data

In [ ]:
# Normalizing the data as KNN is done via a distance measure and we don't want to skew it
scaler = MinMaxScaler()
df_info_knn_normalized = pd.DataFrame(scaler.fit_transform(df_info_knn_encoded), columns = df_info_knn_encoded.columns)

# inspecting new state of the dataframe after normalisation
df_info_knn_normalized

### Now data is prepared, using KNN to impute the missing values
* Experiment with different numbers for nearest neighbours

In [ ]:
# now using KNN to impute the missing values
imputer = KNNImputer(n_neighbors=5)
df_info_knn_imputed = pd.DataFrame(imputer.fit_transform(df_info_knn_normalized),columns = df_info_knn_normalized.columns)

# inspecting new state of the dataset now it no longer has missing values
df_info_knn_imputed

In [ ]:
# checking it no longer has missing values
df_info_knn_imputed.isna().any()

In [ ]:
# looking at sum of missing values
df_info_knn_imputed.isna().sum()

#### Turning these imputed values to usable values

In [ ]:
#Unnormalsing the data
df_info_knn_imputed.columns[1:] #ignoring meter id as that wasn't normalized
df_info_knn_finished = pd.DataFrame(scaler.inverse_transform(df_info_knn_imputed), columns = df_info_knn_imputed.columns) # reversing the normalization

In [ ]:
# viewing the unormalised data
df_info_knn_finished

In [ ]:
#rounding bedrooms to their final value
num_bedrooms=df_info_knn_finished["num_bedrooms"].round() 

In [ ]:
#rounding dwelling type to a real value
df_dwellings=df_info_knn_finished["dwelling_type"].round() 
#reverting the encoding
df_dwellings=df_dwellings.replace(0, "flat").replace(1, "terraced_house").replace(2, "semi_detached_house").replace(3, "detached_house").replace(4, "bungalow")


In [ ]:
#concatenating the meter_id, number of bedrooms, and dwellings columns
df_info_knn_finished = pd.concat([meter_id, num_bedrooms, df_dwellings], axis=1)

In [ ]:
#inspecting the final configuration of the knn imputed values
df_info_knn_finished

## Checking the unique values are the same as the original set

In [ ]:
#looking at the unique values for each column before imputation
print("Unique values before imputation")

print("num_bedrooms unique value count: "+str(df_info_all_meters["num_bedrooms"].nunique()))
print(df_info_all_meters["num_bedrooms"].unique())
print()
print("dwelling_type unique value count: "+str(df_info_all_meters["dwelling_type"].nunique()))
print(df_info_all_meters["dwelling_type"].unique())
print()

In [ ]:
#looking at the unique values for each column before imputation
print("Unique values after imputation\n")

print("num_bedrooms unique value count: "+str(df_info_knn_finished["num_bedrooms"].nunique()))
print(df_info_knn_finished["num_bedrooms"].unique())
print()
print("dwelling_type unique value count: "+str(df_info_knn_finished["dwelling_type"].nunique()))
print(df_info_knn_finished["dwelling_type"].unique())
print()

## Comparing the distribution of unique values that were distributed to the original

In [ ]:
#looking at percentage distribution of values before imputation
print("Distribution of unique values before imputation")
print()

num_bedrooms_percentages = df_info_all_meters['num_bedrooms'].value_counts(normalize=True) * 100
print("num_bedrooms percentages")
print(num_bedrooms_percentages)
print()
print("num_bedrooms totals")
print(df_info_all_meters['num_bedrooms'].value_counts())
print()
print()

dwelling_type_percentages = df_info_all_meters['dwelling_type'].value_counts(normalize=True) * 100
print("dwelling_type percentages")
print(dwelling_type_percentages)
print()
print("dwelling_type totals")
print(df_info_all_meters['dwelling_type'].value_counts())

In [ ]:
#looking at percentage distribution of values after imputation
print("Distribution of unique values before imputation")
print()

num_bedrooms_percentages = df_info_knn_finished['num_bedrooms'].value_counts(normalize=True) * 100
print("num_bedrooms percentages")
print(num_bedrooms_percentages)
print()
print("num_bedrooms totals")
print(df_info_knn_finished['num_bedrooms'].value_counts())
print()
print()

dwelling_type_percentages = df_info_knn_finished['dwelling_type'].value_counts(normalize=True) * 100
print("dwelling_type percentages")
print(dwelling_type_percentages)
print()
print("dwelling_type totals")
print(df_info_knn_finished['dwelling_type'].value_counts())
print()

##### Conclusions:
* Hasn't imputed any meters to be a flat
* Hasn't imputed any meters to have 1 or 5 bedrooms
* Has essentially increased the % divide
    * Higher % represented classes were more likely to be picked
    * Whilst lower % represented classes were less likely to be picked 
* Expected behaviour but is this useful and has it imputed correctly?
* As mentioned already will experiment and see if using these meters or not improves performance more
    * From this we can infer if this was effective or not

## Plotting hists of the imputed addInfo
plotting hists of the imputed addInfo columns before and after imputation to compare their patterns

In [ ]:
# plotting a histogram for each of the imputed columns in the dataset before imputation
for column in df_info_knn_finished.drop(["meter_id"],axis=1).columns:
    df_temp = df_info_all_meters[column]
    ax = df_temp.hist(figsize=(10,10))
    ax.set_xlabel("unique values")
    ax.set_ylabel("count")
    fig = ax.get_figure()
    plt.title(column+" values")
     plt.show(block=False)
    
#     if(os.path.exists(f"../EDA/plots/addInfo/Pre-Imputation/{column}_preimp_hist.png")==False):
#         fig.savefig(f"../EDA/plots/addInfo/Pre-Imputation/{column}_preimp_hist.png")
    fig.savefig(f"../EDA/plots/addInfo/Pre-Imputation/{column}_preimp_hist.png")
    
    plt.close(fig)

In [ ]:
# plotting a histogram for each of the columns in the dataset after imputation
for column in df_info_knn_finished.drop(["meter_id"],axis=1).columns:
    df_temp = df_info_knn_finished[column]
    ax = df_temp.hist(figsize=(10,10))
    ax.set_xlabel("unique values")
    ax.set_ylabel("count")
    fig = ax.get_figure()
    plt.title(column+" values")
    plt.show(block=False)
    
#     if(os.path.exists(f"../EDA/plots/addInfo/Imputed/{column}_imp_hist.png")==False):
#         fig.savefig(f"../EDA/plots/addInfo/Imputed/{column}_imp_hist.png")
    fig.savefig(f"../EDA/plots/addInfo/Imputed/{column}_imp_hist.png")
        
    plt.close(fig)

## OneHot Encoding Ready for the RNN to make use of

In [ ]:
#encoding the categorical values via one hot encoding
cat_variables = df_info_knn_finished[df_info_knn_finished.loc[:,df_info_knn_finished.columns!="meter_id"].select_dtypes(include=['object']).columns] # getting the categorical columns (just 'dwelling_type')
cat_dummies = pd.get_dummies(cat_variables, drop_first=True, dummy_na=True) # getting dummies, including a separate one for na

#setting nan rows and droppoing nan column
cat_dummies.loc[cat_dummies.dwelling_type_nan==1, ["dwelling_type_detached_house","dwelling_type_flat","dwelling_type_semi_detached_house","dwelling_type_terraced_house"]]=np.nan
cat_dummies=cat_dummies.drop("dwelling_type_nan",axis=1)
cat_dummies

In [ ]:
df_info_knn_finished

In [ ]:
# dropping the original columns then adding the dummy values
#df_info_knn_finished = df_info_knn_finished.drop(df_info_knn_finished.loc[:,df_info_knn_finished.columns!="meter_id"].select_dtypes(include=['object']).columns, axis=1)
df_info_knn_finished = pd.concat([df_info_knn_finished, cat_dummies], axis=1)
# inspecting new state of the dataframe with the dummy variables
df_info_knn_finished

## Generating correlation plot of the features to make sure there is minimal redundancy

In [ ]:
#generating the correlation matrix
corr_info = df_info_knn_finished.corr()

In [ ]:
#generating the correlation heat map
plt.figure(figsize=(10, 10))
sns.heatmap(corr_info, cmap="coolwarm", annot=True, square=True)

In [ ]:
# for each feature on the x axis
for i in range(corr_info.shape[0]):
    # for each feature after i
    for j in range(i+1, corr_info.shape[0]):
        #check if their correlation is above a threshold % (50%)
        if abs(corr_info.iloc[i,j]) >= 0.5:
            print("\nConsider ",corr_info.columns[j]," for removal as it has a high correlation with ",corr_info.columns[i]," of",round(corr_info.iloc[i,j],2),"\n")
            #set column j to be removed 
            if columns[j]:
                columns[j+1] = False #+1 for meter id
        else: 
            print("Keeping ",corr_info.columns[i]," and ",corr_info.columns[j]," have a correlation of ", round(corr_info.iloc[i,j],2))

##### Features don't have a particularly high correlation; no need to remove them based upon this

## Saving the finished AddInfo

In [ ]:
# if(os.path.exists(f"../Data/Preprocessed_Data/addInfo_knn.pkl")==False):
#         df_info_knn_finished.to_pickle(f"../Data/Preprocessed_Data/addInfo_knn.pkl")
df_info_knn_finished.to_pickle(f"../Data/Preprocessed_Data/addInfo_knn.pkl")